**Task 07: Querying RDF(s)**

In [ ]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 


Read the RDF file

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")
for s,p,o in g:
  print(s,p,o)

http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#UPM http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#University
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf ht

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [25]:
#RDFLib 
ns=Namespace("http://somewhere#")

for s in g.subjects(RDFS.subClassOf, ns.Person):
  print(s)
  for r in g.subjects(RDFS.subClassOf,s):
    print(r)


http://somewhere#Researcher
http://somewhere#PhDstudent


In [82]:
# SPARQL 
from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT 
    ?x
  WHERE { 
    ?x  rdfs:subClassOf/rdfs:subClassOf*  ns:Person. 
  }
  ''', initNs = { "ns": ns,"rdfs":RDFS}
)
for r in g.query(q1):
  print(r)


(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [81]:
#RDFLib
sub=[]
for s in g.subjects(RDF.type,ns.Person):
  print(s)
  sub.append(s)

for s in g.subjects(RDF.type/RDFS.subClassOf,ns.Person):
  print(s) 
  sub.append(s)


http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith


In [80]:

#SPARQL 

q1 = prepareQuery('''
  SELECT 
    ?ind
  WHERE { 
    ?ind a/rdfs:subClassOf*  ns:Person. 
  }
  ''', initNs = { "ns": ns,"rdfs":RDFS}
)
for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [79]:
#RDFLib
for i in range(len(sub)):
  for s,p,o in  g.triples((sub[i],None,None)):
    if(p!=RDF.type):
      print(s,p)


http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given


In [78]:
#SPARQL
q1 = prepareQuery('''
  SELECT 
    ?ind ?p 
  WHERE { 
    ?ind a/rdfs:subClassOf*  ns:Person. 
    ?ind ?p  ?value
    FILTER(?p!=rdf:type)
  }
  ''', initNs = { "ns": ns,"rdfs":RDFS}
)
for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.URIRef